In [26]:
import numpy as np
import pandas as pd
import requests
import os
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

#Personal Header
headers = {'user-agent': 'Muhurto Personal Project (bur6yx@virginia.edu)'}

#Profile Id for Valorant
profileid = "Muhurto-7071"

r = requests.get(f"https://u.gg/val/profile/{profileid}", headers = headers)




e35d9f3f-1039-4269-ba3a-33be025c8cb4
<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><title>Muhurto#7071&#x27;s Profile - Overview</title><meta name="viewport" content="width=device-width"/><meta name="description" content="Best Peeks from the Best Data. Riot-partnered U.GG provides the best Valorant stats to help you up your game with peek maps, agent tierlists and more."/><meta property="description" content="Best Peeks from the Best Data. Riot-partnered U.GG provides the best Valorant stats to help you up your game with peek maps, agent tierlists and more."/><meta property="og:site_name" content="U.GG Valorant"/><meta property="og:type" content="website"/><meta property="og:title" content="Muhurto#7071&#x27;s Profile - Overview"/><meta property="og:description" content="Best Peeks from the Best Data. Riot-partnered U.GG provides the best Valorant stats to help you up your game with peek maps, agent tierlists and more."/><meta property="og:image" content="https://static.b

In [302]:
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service = service)

driver.get(f"https://u.gg/val/profile/{profileid}")

time.sleep(7)

while True:
    try:
        input_element = driver.find_element(By.CSS_SELECTOR,
                                            ".flex.items-center.justify-center.group.rounded-sm.px-4.py-2\\.5.text-\\[13px\\].font-bold.dark\\:bg-lavender-500.dark\\:hover\\:bg-lavender-600.dark\\:text-white.dark\\:ui-toggled\\:bg-lavender-500.dark\\:ui-toggled\\:font-bold.leading-none")
        input_element.click()
        time.sleep(2)
    except Exception as e:
        print("No More Matches to Load", e)
        break


time.sleep(7)


page_source = driver.page_source

driver.quit()


No More Matches to Load Message: no such element: Unable to locate element: {"method":"css selector","selector":".flex.items-center.justify-center.group.rounded-sm.px-4.py-2\.5.text-\[13px\].font-bold.dark\:bg-lavender-500.dark\:hover\:bg-lavender-600.dark\:text-white.dark\:ui-toggled\:bg-lavender-500.dark\:ui-toggled\:font-bold.leading-none"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6F0D1FB05+28789]
	(No symbol) [0x00007FF6F0C886E0]
	(No symbol) [0x00007FF6F0B2592A]
	(No symbol) [0x00007FF6F0B7930E]
	(No symbol) [0x00007FF6F0B795FC]
	(No symbol) [0x00007FF6F0BC28A7]
	(No symbol) [0x00007FF6F0B9F47F]
	(No symbol) [0x00007FF6F0BBF654]
	(No symbol) [0x00007FF6F0B9F1E3]
	(No symbol) [0x00007FF6F0B6A938]
	(No symbol) [0x00007FF6F0B6BAA1]
	GetHandleVerifier [0x00007FF6F105933D+3410093]
	GetHandleVerifier [0x

In [304]:

defeats = soup.find_all("span", class_ = 'dark:text-accent-red-500 inline-block mr-2 text-md font-bold leading-[17px]')
victories = soup.find_all("span", class_ = "dark:text-accent-green-300 inline-block mr-2 text-md font-bold leading-[17px]")
matches = soup.find_all("div", class_ = "flex flex-col justify-around select-none w-full")

#Just iterate this first number to scrape through and work with all the information here


playedagents = []
roundswon = []
roundslost = []
numkills = []
numdeaths = []
numassists = []
accuracy = []
mapplayed = []
acs = []
rank = []
relativerank = []


for i in range(len(matches)):
    #Played Agents works for the first 20 matches!
    playedagents.append(matches[i].find_all("div", class_ = "flex items-center mr-3")[0].find("span").attrs['class'][-1].split("-")[-1])
    
    
    roundswon.append(matches[i].find_all("div", class_ = "font-bold text-sm")[0].find_all("span")[0].text)
    
    
    
    roundslost.append(matches[i].find_all("div", class_ = "font-bold text-sm")[0].find_all("span")[2].text)
    numkills.append(matches[i].find_all("div", class_ = "dark:text-lavender-300")[0].find_all("span")[0].text)
    numdeaths.append(matches[i].find_all("div", class_ = "dark:text-lavender-300")[0].find_all("span")[1].text)
    numassists.append(matches[i].find_all("div", class_ = "dark:text-lavender-300")[0].find_all("span")[2].text)
    mapplayed.append(matches[i].find_all("div", class_= "flex items-center xs:relative w-full xs:w-[100px] h-full bg-contain bg-center rounded-sm")[0].text)
    acs.append(matches[i].find_all("div", class_ = "text-xxs dark:text-lavender-50")[0].text.split()[0])
    relativerank.append(matches[i].find_all("div", class_ = "flex flex-col h-full justify-between items-center text-xs xs:ml-5")[0].find_all("span")[3].text)
    rank.append(matches[i].find_all("div", class_ = "flex items-center nowrap whitespace-nowrap")[0].text)
    accuracy.append(matches[i].find_all("div", class_ = "flex items-center font-semibold text-xs")[0].text)
    # Same weird thing with map, the span background is a little different. 

pdata = pd.DataFrame(
    {
        "Agent": playedagents,
        "RoundWins": roundswon,
        "RoundLosses": roundslost,
        "Kills": numkills,
        "Deaths": numdeaths,
        "Assists": numassists,
        "Accuracy": accuracy,
        "Map": mapplayed,
        "CombatScore": acs,
        "MatchRank": relativerank,
        "CompetitiveRank": rank
    }
)
pdata

,Agent,RoundWins,RoundLosses,Kills,Deaths,Assists,Accuracy,Map,CombatScore,MatchRank,CompetitiveRank
0,killjoy,11,13,20,19,10,13.6%,Ascent,244,4th,Silver 2
1,breach,6,13,15,15,7,13.1%,Abyss,237,4th,Silver 2
2,harbor,7,13,12,17,8,22.6%,Bind,207,4th,Silver 2
3,killjoy,13,4,7,11,8,6.9%,Ascent,155,7th,Silver 2
4,harbor,13,11,15,17,8,24.5%,Split,173,7th,Silver 2
...,...,...,...,...,...,...,...,...,...,...,...
245,kayo,13,4,8,8,8,19.2%,Split,125,7th,Silver 2
246,killjoy,13,9,18,17,5,7.3%,Breeze,229,3rd,Silver 2
247,killjoy,13,3,18,11,2,7.3%,Ascent,306,2nd,Silver 2
248,harbor,13,8,19,13,5,23.1%,Breeze,240,3rd,Silver 1
